In [1]:
import os
import json
from dotenv import load_dotenv
import openai
from prompts import *
import numpy as np

from Chat import AgentClient
from Toolbox import ImageProcessingToolBoxes

# Set the environment variable for the OpenAI API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

In [2]:
test_image_path = './test-images/a0005-jn_2007_05_10__564.tiff'
output_dir_path = f'./test-output/{os.path.splitext(os.path.basename(test_image_path))[0]}'
image_process = ImageProcessingToolBoxes(image_path=test_image_path, output_dir_name=output_dir_path, debug=True, save_high_resolution=True)
chat_client = AgentClient(api_key=api_key, toolbox_instance=image_process, debug=False)
global_style = None

In [3]:
# 给每一步编号，并回到某一步, 改prompt
# 加入HIsto

In [3]:
# 分析图像的内容是什么
_ = chat_client.agent_interaction(
    system_prompt=image_content_analyzer_prompt()
)
# 分析图像的直方图现状
histo_style = chat_client.agent_interaction(
    system_prompt=image_global_histo_style_analyzer_prompt(),
    histo_image=True
)
# 根据图片分析，提出一个总体的色彩校正目标
global_style = chat_client.agent_interaction(
    system_prompt=image_global_style_analyzer_prompt(histo_instruction=histo_style)
)
# 根据总体色彩校正目标和可用的图像处理手段，提出初步的校正概念
_ = chat_client.agent_interaction(
    system_prompt=global_retouching_concept_prompt(function_aspects=image_process.get_function_short_description(), global_instruction=global_style)
)
# 并得到矫正计划
_ = chat_client.agent_get_plan(
    system_prompt=retouching_planing_system_prompt(),
    user_prompt=retouching_planing_user_prompt(image_process.get_function_names(), global_instruction=global_style)
)



`func_to_return_responses`, response: 

**Image Content Analysis:**
The photograph captures a narrow alleyway framed by darkened silhouettes of buildings on the sides. The main focus lies on an opening at the end of the alley where a lighter, sunlit area reveals a yellow building with green shutters. Two no-parking signs are visible on either side of the alley, accompanied by antique-style street lamps. The sky, appearing bright and blue, provides contrast to the shadowy alley.

**Photographic Value Evaluation:**
The composition relies heavily on dramatic contrast, with deep shadows in the foreground buildings leading the eye through the passage to the well-lit background. The shooting angle emphasizes the height and depth of the alley, creating a sense of perspective. The interplay between light and shadow adds a mysterious and captivating quality to the photograph. While the overall balance is compelling, the shadowed foreground may benefit from subtle brightening to enhance detail

In [4]:
while image_process.processing_plan:
    current_function = image_process.processing_plan[0]

    # Histo

    if image_process.plan_status != current_function:  # 如果正在执行的function和现在的一样，说明是重新执行的。否则是重新开始
        image_process.set_plan_status(current_function)
        image_process.set_satisfactory_status(False)  # 开始新的function默认不满意

        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_execute_user_prompt(current_function),
            current_function_name=current_function)
    else:
        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_re_execute_user_prompt(current_function),
            current_function_name=current_function)

    chat_client.agent_reflection_plan(
        system_prompt=retouching_reflection_system_prompt(),
        combine_image=False
    )

    if image_process.satisfactory_status:
        image_process.finish_current_plan()



`func_to_return_responses`, response: 

**Analysis of Main Content**
The photo features two young children in an outdoor setting. One child, wearing a blue hoodie, is engaged in eating while the other, in a multi-colored top, looks on attentively. There's an outdoor chair visible in the background, with green foliage creating a natural backdrop. The children appear to be in a casual, playful scenario.

**Evaluation of Photographic Value**
The image captures a candid moment from a frontal angle with a tight composition, focusing on the children. The lighting is natural, and even though it seems slightly overcast, it softens the shadows. The bright colors of the children’s clothing contrast the muted greens of the background, creating a visually pleasing balance.

**Speculation on Emotions and Artistic Values**
This image evokes feelings of innocence and curiosity, capturing a tender moment of interaction between the children. The simplicity and spontaneity lend an authentic and relata

In [16]:
image_process.check_history_messages()


Message 1 | Role: assistant
--------------------------------------------------------------------------------
### Analysis of Main Content
The photo features a woman standing under a portable canopy, holding an orange foam noodle and some tape. The setting appears to be a sunny outdoor area, possibly a park or backyard, with greenery in the background and a small paved area beneath her. She's casually dressed in a loose-fitting pink shirt. The image is likely candid, capturing her in action, perhaps preparing for an outdoor event or activity.

### Evaluation of Photographic Value
The image is shot at eye level, which provides a natural view of the subject. The lighting is bright and sunny, casting clear shadows on the ground, which adds depth to the photo. The use of natural light enhances the colors, making them vibrant but also causing some harsh shadows and highlights. The composition is straightforward, focusing primarily on the subject, with the canopy framing her and serving as a

In [17]:
sum(chat_client.total_tokens)

67439

In [5]:
client = openai.OpenAI(api_key=api_key)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
messages.append({"role": "user", "content": "i think it is order_12345"})

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    tools=tools
)

In [8]:
response.usage.total_tokens

159